In [1]:
# mpnn design all the looped poses

from glob import glob

with open("03_pair_looped/03_paired_looped_poses.list", 'w') as f:
    for fname in sorted(glob("/home/broerman/crispy_shifty/projects/crispy_shifty_dimers/round_3/design/03_pair_looped/decoys/*/*.pdb.bz2")):
        f.write(fname + "\n")

In [6]:
import sys
sys.path.insert(0, '/home/broerman/crispy_shifty/')

from crispy_shifty.utils.io import gen_array_tasks

simulation_name = "04_two_state"
design_list_file = "/home/broerman/crispy_shifty/projects/crispy_shifty_dimers/round_3/design/03_pair_looped/03_paired_looped_poses.list"
output_path = f"/home/broerman/crispy_shifty/projects/crispy_shifty_dimers/round_3/design/{simulation_name}"

options = " ".join(
    [
        "out:level 200",
    ]
)

extra_kwargs = {
    "num_sequences": "10",
    "batch_size": "10",
    "mpnn_temperature": 0.2,
    "mpnn_design_area": "full",
    "mpnn_betas": "scan",
}

gen_array_tasks(
    distribute_func="crispy_shifty.protocols.mpnn.mpnn_dimers",
    design_list_file=design_list_file,
    output_path=output_path,
    queue="short",
    memory="12G",
    nstruct=1,
    nstruct_per_task=5,
    options=options,
    extra_kwargs=extra_kwargs,
    simulation_name=simulation_name,
)

Run the following command with your desired environment active:
sbatch -a 1-1012 /home/broerman/crispy_shifty/projects/crispy_shifty_dimers/round_3/design/04_two_state/run.sh


#### Collect scorefiles of the MPNN designed paired states and concatenate ####

In [ ]:
import os
from crispy_shifty.utils.io import collect_score_file

simulation_name = "04_two_state"
output_path = f"/home/broerman/crispy_shifty/projects/crispy_shifty_dimers/round_3/design/{simulation_name}"

if not os.path.exists(os.path.join(output_path, "scores.json")):
    collect_score_file(output_path, "scores")

#### Load resulting concatenated scorefile ####

In [ ]:
import sys
sys.path.insert(0, '/home/broerman/crispy_shifty/')
from crispy_shifty.utils.io import parse_scorefile_linear

scores_df = parse_scorefile_linear(os.path.join(output_path, "scores.json"))
scores_df = scores_df.convert_dtypes()

100%|██████████| 861/861 [00:03<00:00, 229.05it/s]


#### Save individual fastas ####

In [ ]:
from crispy_shifty.utils.io import df_to_fastas

scores_df = df_to_fastas(scores_df, prefix="mpnn_seq")

100%|██████████| 861/861 [00:02<00:00, 374.08it/s]


#### Save a list of outputs ####

In [ ]:
from tqdm.auto import tqdm
with open(os.path.join(output_path, "mpnn_paired_states.list"), "w") as f:
    for path in tqdm(scores_df.index):
        print(path, file=f)

100%|██████████| 861/861 [00:00<00:00, 1119434.51it/s]


#### Concat the pdb.bz2 and fasta paths into a single list, for Superfold reasons ####

In [ ]:
with open(os.path.join(output_path, "mpnn_paired_states.pair"), "w") as f:
    for path in tqdm(scores_df.index):
        line = path + "____" + path.replace("decoys", "fastas").replace("pdb.bz2", "fa")
        print(line, file=f)

100%|██████████| 861/861 [00:00<00:00, 745057.92it/s]


Go fold on perlmutter